## Final Project Submission

Please fill out:
* Student name: 
* Student pace: self paced / part time / full time
* Scheduled project review date/time: 
* Instructor name: 
* Blog post URL:


In [ ]:
# Your code here - remember to use markdown cells for comments as well!

In [8]:
!pip install spacy

smart-open 1.8.0 requires bz2file, which is not installed.
pexpect 4.6.0 requires ptyprocess>=0.5, which is not installed.
nltk 3.4 requires singledispatch, which is not installed.
You are using pip version 10.0.1, however version 19.1.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [9]:
import spacy
nlp = spacy.load('en')

OSError: [E050] Can't find model 'en'. It doesn't seem to be a shortcut link, a Python package or a valid path to a data directory.

In [2]:
# with open(newfile, 'w') as outfile, open(oldfile, 'r', encoding='utf-8') as infile:

with open('chapter1.txt', 'r', encoding = 'utf-8') as f:
    chapter1 = f.read()
len(chapter1)

93451

In [10]:
chapter1[:1000]

'Once upon a time and a very good time it was there was a moocow coming\ndown along the road and this moocow that was coming down along the road\nmet a nicens little boy named baby tuckoo....\n\nHis father told him that story: his father looked at him through a\nglass: he had a hairy face.\n\nHe was baby tuckoo. The moocow came down the road where Betty Byrne\nlived: she sold lemon platt.\n\n    O, the wild rose blossoms\n    On the little green place.\n\n\nHe sang that song. That was his song.\n\n    O, the green wothe botheth.\n\n\nWhen you wet the bed, first it is warm then it gets cold. His mother\nput on the oilsheet. That had the queer smell.\n\nHis mother had a nicer smell than his father. She played on the piano\nthe sailor’s hornpipe for him to dance. He danced:\n\n    Tralala lala,\n    Tralala tralaladdy,\n    Tralala lala,\n    Tralala lala.\n\n\nUncle Charles and Dante clapped. They were older than his father and\nmother but uncle Charles was older than Dante.\n\nDante had

In [15]:
chapter1_lines = chapter1.split('.')

In [17]:
len(chapter1_lines)

1118

In [3]:
with open('chapter2.txt', 'r', encoding = 'utf-8') as f:
    chapter2 = f.read()
len(chapter2)

82955

In [11]:
chapter2[:1000]

"{\\rtf1\\ansi\\ansicpg1252\\cocoartf1671\\cocoasubrtf400\n{\\fonttbl\\f0\\fswiss\\fcharset0 Helvetica;}\n{\\colortbl;\\red255\\green255\\blue255;}\n{\\*\\expandedcolortbl;;}\n\\margl1440\\margr1440\\vieww10800\\viewh8400\\viewkind0\n\\pard\\tx720\\tx1440\\tx2160\\tx2880\\tx3600\\tx4320\\tx5040\\tx5760\\tx6480\\tx7200\\tx7920\\tx8640\\pardirnatural\\partightenfactor0\n\n\\f0\\fs24 \\cf0 Uncle Charles smoked such black twist that at last his nephew suggested\\\nto him to enjoy his morning smoke in a little outhouse at the end of\\\nthe garden.\\\n\\\n\\'97Very good, Simon. All serene, Simon, said the old man tranquilly.\\\nAnywhere you like. The outhouse will do me nicely: it will be more\\\nsalubrious.\\\n\\\n\\'97Damn me, said Mr Dedalus frankly, if I know how you can smoke such\\\nvillainous awful tobacco. It\\'92s like gunpowder, by God.\\\n\\\n\\'97It\\'92s very nice, Simon, replied the old man. Very cool and mollifying.\\\n\\\nEvery morning, therefore, uncle Charles repaired to hi

In [18]:
len(chapter2.split('.'))

738

In [4]:
with open('chapter3.txt', 'r', encoding = 'utf-8') as f:
    chapter3 = f.read()
len(chapter3)

92923

In [12]:
chapter3[:1000]

'{\\rtf1\\ansi\\ansicpg1252\\cocoartf1671\\cocoasubrtf400\n{\\fonttbl\\f0\\fswiss\\fcharset0 Helvetica;}\n{\\colortbl;\\red255\\green255\\blue255;}\n{\\*\\expandedcolortbl;;}\n\\margl1440\\margr1440\\vieww10800\\viewh8400\\viewkind0\n\\pard\\tx720\\tx1440\\tx2160\\tx2880\\tx3600\\tx4320\\tx5040\\tx5760\\tx6480\\tx7200\\tx7920\\tx8640\\pardirnatural\\partightenfactor0\n\n\\f0\\fs24 \\cf0 Chapter III\\\n\\\n\\\nThe swift December dusk had come tumbling clownishly after its dull day\\\nand as he stared through the dull square of the window of the\\\nschoolroom he felt his belly crave for its food. He hoped there would\\\nbe stew for dinner, turnips and carrots and bruised potatoes and fat\\\nmutton pieces to be ladled out in thick peppered flour-fattened sauce.\\\nStuff it into you, his belly counselled him.\\\n\\\nIt would be a gloomy secret night. After early nightfall the yellow\\\nlamps would light up, here and there, the squalid quarter of the\\\nbrothels. He would follow a devious c

In [19]:
len(chapter3.split('.'))

747

In [5]:
with open('chapter4.txt', 'r', encoding = 'utf-8') as f:
    chapter4 = f.read()
len(chapter4)

54328

In [13]:
chapter4[:1000]

'{\\rtf1\\ansi\\ansicpg1252\\cocoartf1671\\cocoasubrtf400\n{\\fonttbl\\f0\\fswiss\\fcharset0 Helvetica;}\n{\\colortbl;\\red255\\green255\\blue255;}\n{\\*\\expandedcolortbl;;}\n\\margl1440\\margr1440\\vieww10800\\viewh8400\\viewkind0\n\\pard\\tx720\\tx1440\\tx2160\\tx2880\\tx3600\\tx4320\\tx5040\\tx5760\\tx6480\\tx7200\\tx7920\\tx8640\\pardirnatural\\partightenfactor0\n\n\\f0\\fs24 \\cf0 Sunday was dedicated to the mystery of the Holy Trinity, Monday to the\\\nHoly Ghost, Tuesday to the Guardian Angels, Wednesday to Saint Joseph,\\\nThursday to the Most Blessed Sacrament of the Altar, Friday to the\\\nSuffering Jesus, Saturday to the Blessed Virgin Mary.\\\n\\\nEvery morning he hallowed himself anew in the presence of some holy\\\nimage or mystery. His day began with an heroic offering of its every\\\nmoment of thought or action for the intentions of the sovereign pontiff\\\nand with an early mass. The raw morning air whetted his resolute piety;\\\nand often as he knelt among the few wo

In [6]:
with open('chapter5.txt', 'r', encoding = 'utf-8') as f:
    chapter5 = f.read()
len(chapter5)

152007

In [14]:
chapter5[:1000]

"{\\rtf1\\ansi\\ansicpg1252\\cocoartf1671\\cocoasubrtf400\n{\\fonttbl\\f0\\fswiss\\fcharset0 Helvetica;}\n{\\colortbl;\\red255\\green255\\blue255;}\n{\\*\\expandedcolortbl;;}\n\\margl1440\\margr1440\\vieww10800\\viewh8400\\viewkind0\n\\pard\\tx720\\tx1440\\tx2160\\tx2880\\tx3600\\tx4320\\tx5040\\tx5760\\tx6480\\tx7200\\tx7920\\tx8640\\pardirnatural\\partightenfactor0\n\n\\f0\\fs24 \\cf0 He drained his third cup of watery tea to the dregs and set to chewing\\\nthe crusts of fried bread that were scattered near him, staring into\\\nthe dark pool of the jar. The yellow dripping had been scooped out like\\\na boghole and the pool under it brought back to his memory the dark\\\nturfcoloured water of the bath in Clongowes. The box of pawn tickets at\\\nhis elbow had just been rifled and he took up idly one after another in\\\nhis greasy fingers the blue and white dockets, scrawled and sanded and\\\ncreased and bearing the name of the pledger as Daly or MacEvoy.\\\n\\\n\\\n  1 Pair Buskins.\\

In [ ]:
#doc = nlp(open("pg345.txt").read().decode('utf8'))